Using the 2025 Australian GP as an example, the loop below creates a dataframe of each team's average straight and corner speeds.

In [ ]:
import fastf1
import pandas as pd

fastf1.Cache.enable_cache('/Users/tracey/Documents/Python/') #replace with ur own folder path to make a cache for faster loading

session = fastf1.get_session(2025, 'Australian', 'R')
session.load()

race_teams = session.laps['Team'].unique()
print(race_teams)

all_teams_features = []

for team_name in race_teams:
    laps = session.laps.pick_teams(team_name)

    if laps.empty:
        continue
    
    lap_features = []

    for _, lap in laps.iterrows():
        #test: take sector 1 as straight, sector 2+3 as corners
        avg_straight_speed = lap['SpeedI1']  # sector 1 speed
        max_speed = lap['SpeedST']           # speed trap
        avg_corner_speed = (lap['SpeedI2'] + lap['SpeedFL']) / 2  # sector 2 + corners
        min_corner_speed = min(lap['SpeedI2'], lap['SpeedFL'])
        
        lap_features.append({
            'AvgStraightSpeed': avg_straight_speed,
            'MaxSpeed': max_speed,
            'AvgCornerSpeed': avg_corner_speed,
            'MinCornerSpeed': min_corner_speed
        })

    lap_features_df = pd.DataFrame(lap_features)  #per lap

#aggregated across all laps for the overall team performance
    team_features_df = lap_features_df.mean().to_frame().T 
    team_features_df['Team'] = team_name
    team_features_df = team_features_df[['Team'] + [col for col in team_features_df.columns if col != 'Team']] #reorder
    all_teams_features.append(team_features_df)

final_df = pd.concat(all_teams_features, ignore_index = True)
print(final_df)